In [1]:
import pandas as pd
import re
biggerfile="/Users/irffanalahi/Research/Research_code/gitignorefolder/metpostprocess/troubleshhoting/BLthirteen_all_matrixCin_nr0.5_imputed_rowmean_head1000.txt_bg_intersectedbg/BLthirteen_all_matrixCin_nr0.5_imputed_g1_M0_3_g2_others_36.txt_bg_cin.txt"
phenfile="/Users/irffanalahi/Research/Research_code/gitignorefolder/metpostprocess/troubleshhoting/BL13PhenoClass.txt"
outfile="/Users/irffanalahi/Research/Research_code/gitignorefolder/metpostprocess/troubleshhoting/out4.txt"
biggerdf=pd.read_csv(biggerfile,sep="\t",header=None,skiprows=1)
pheno=pd.read_csv(phenfile,sep="\t", header=None, index_col=0)

biggerdf.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,chr1:16082,1.000000,0.807000,1.000000,0.938000,0.940000,0.798500,0.952333,0.964500,0.659000,...,chr1,16082,16244,1.00000,-0.100017,4,0.126100,0.28833,0.73300,0.83302
1,chr1:16139,0.879333,0.870667,0.954667,0.935667,0.990333,0.918667,1.000000,0.966667,0.909667,...,chr1,16082,16244,1.00000,-0.100017,4,0.126100,0.28833,0.73300,0.83302
2,chr1:16141,0.837000,0.832333,0.966667,0.914667,0.977000,0.867333,1.000000,0.911667,0.855000,...,chr1,16082,16244,1.00000,-0.100017,4,0.126100,0.28833,0.73300,0.83302
3,chr1:16243,0.654000,0.641667,0.765667,0.697667,0.651667,0.743333,0.725667,0.707000,0.567000,...,chr1,16082,16244,1.00000,-0.100017,4,0.126100,0.28833,0.73300,0.83302
4,chr1:102933,0.968333,0.884333,0.965000,0.910000,0.101500,0.902667,0.934667,0.900000,0.939333,...,chr1,102933,103065,0.46632,0.117913,4,0.006395,0.01516,0.98358,0.86567


In [2]:
def headerfix(existingcolnames):

    m = re.search('g1_(.+?)_\d+_g2', biggerfile)
    if m:
        found = m.group(1)

    else:
        print("cell type not found", biggerfile)
        print("exiting")

        sys.exit(1)

    celltype = found
    
  
    
    
    newcolnames=["chrom","start","end","q","diff","#cpg","p(MWU)","p(2dks)",celltype+"dup","others"]
    return celltype,newcolnames

def comparmentwisdiff(newdf,celltype):
    cells=(pheno.index).tolist()

    compartmendeltalist=[]
    for cell in cells:
        if cell!=celltype:
            tempdelta=celltype+"-"+cell
            newdf[tempdelta]=newdf.iloc[:,8]-newdf[cell]

            compartmendeltalist.append(tempdelta)




    newdf['maxCompartmentwisedelta']=newdf[compartmendeltalist].max(axis=1)
    newdf['minCompartmentwiseDelta']=newdf[compartmendeltalist].min(axis=1)







    return newdf

In [3]:
with open (biggerfile) as f:
    line=f.readline()
    line=line.split("\n")
    colnames=line[0].split("\t")

In [4]:
thatcell,furthercols=headerfix(colnames)
totalcols=colnames+furthercols

biggerdf.columns=totalcols

biggerdf.head()
biggerdf=biggerdf.drop(['position','shouldbechrom', 'pos'],axis=1)


In [5]:
grouped=biggerdf.groupby(furthercols)

groupedmean=grouped.mean()
groupedcount=grouped.size()


groupedmean=pd.concat([groupedmean,groupedcount],axis=1)
groupedmean.rename(columns={0:"mycpgcount"},inplace=True,errors='raise')


groupedmean=groupedmean.reset_index()






compartdf=comparmentwisdiff(groupedmean,thatcell)

compartdf.rename(columns={thatcell+"dup":thatcell},inplace=True,errors='raise')


duplicateRowsDF = compartdf[compartdf.duplicated(['chrom','start','end'])]

if duplicateRowsDF.shape[0]>0:
    print("Duplicate Rows:", duplicateRowsDF, sep='\n')

compartdf.to_csv(outfile,sep="\t",index=False)

